In [1]:
# 这个文件主要是为了修改DTBS的B区中  xr\xt1\xt2\qr\qt1\qt2
# 是更新A区qs、xx信息之前的准备工作

import pandas as pd
import numpy as np
import pickle
import time 
import os
from datetime import datetime
from operator import itemgetter

data_folder_path = '../'
raw_data_path = "../raw_data"
storage_path = "../database_storage"
daily_path = "../daily"
DTBS_path = os.path.join(storage_path, "DTBS.pkl")
DCBS_path = os.path.join(storage_path, "DCBS.pkl")
with open(DTBS_path, 'rb') as f:  
    DTBS = pickle.load(f)

In [3]:
from WindPy import w

need_add_xxqs_bond_code_list = []

for bond_code,informations in DTBS['B'].items():
    if 'xr' not in informations.keys():
        need_add_xxqs_bond_code_list.append(bond_code)

print(need_add_xxqs_bond_code_list)
"""
# 这里是需要更新的代码列表，要需要更改，通过wind获取下面这些转债的强赎和下修的信息
need_add_xxqs_bond_code_list = ['118035.SH','123199.SZ','123198.SZ','127086.SZ',
                                '111014.SH','123200.SZ','127087.SZ','123201.SZ',
                                '123203.SZ','127088.SZ','118036.SH']
"""

# 这个时间可以不改，理论上转债的条款不会发生变化
date = datetime.today().strftime('%Y-%m-%d')

w.start()

Wind_qt1 = w.wsd(need_add_xxqs_bond_code_list, "clause_calloption_redeemspan", date, date, "").Data
Wind_qt2 = w.wsd(need_add_xxqs_bond_code_list, "clause_calloption_redeemmaxspan", date, date, "").Data
Wind_qr = w.wsd(need_add_xxqs_bond_code_list, "clause_calloption_triggerproportion", date, date, "").Data

Wind_xt1 = w.wsd(need_add_xxqs_bond_code_list, "clause_reset_resettimespan", date, date, "").Data
Wind_xt2 = w.wsd(need_add_xxqs_bond_code_list, "clause_reset_resetmaxtimespan", date, date, "").Data
Wind_xr = w.wsd(need_add_xxqs_bond_code_list, "clause_reset_resettriggerratio", date, date, "").Data

print(Wind_qt1, '\n', Wind_qt2, '\n', Wind_qr, '\n', Wind_xt1, '\n', Wind_xt2, '\n', Wind_xr)

for bond_code_index in range(len(need_add_xxqs_bond_code_list)):
    bond_code = need_add_xxqs_bond_code_list[bond_code_index]
    
    DTBS['B'][bond_code]['qt1'] = Wind_qt1[0][bond_code_index]
    DTBS['B'][bond_code]['qt2'] = Wind_qt2[0][bond_code_index]
    DTBS['B'][bond_code]['qr'] = Wind_qr[0][bond_code_index]
    DTBS['B'][bond_code]['xt1'] = Wind_xt1[0][bond_code_index]
    DTBS['B'][bond_code]['xt2'] = Wind_xt2[0][bond_code_index]
    DTBS['B'][bond_code]['xr'] = Wind_xr[0][bond_code_index]

print(DTBS['B']['111014.SH'])

[]
[['CWSDService:invalid windcodes.']] 
 [['CWSDService:invalid windcodes.']] 
 [['CWSDService:invalid windcodes.']] 
 [['CWSDService:invalid windcodes.']] 
 [['CWSDService:invalid windcodes.']] 
 [['CWSDService:invalid windcodes.']]
{'ipo': '2023-07-13', 'dld': '2046-01-01', 'cn': 'N李子转', 'sc': '605337.SH', 'sn': '李子园', 'cat1': nan, 'cat2': nan, 'qt1': 15.0, 'qt2': 30.0, 'qr': 130.0, 'xt1': 15.0, 'xt2': 30.0, 'xr': 80.0}


In [4]:
# 保存上面的这些信息的更改

f_save = open(DTBS_path, 'wb')
pickle.dump(DTBS, f_save)
f_save.close()